&copy;Copyright 2017 Shuang Wu<br>
cite from the Neural Networks and Deep Learning book by Michael Nielsen http://neuralnetworksanddeeplearning.com <br>
Learning notes

## CH2<br>
## How the backpropagation algorithm works<br>
<strong>backpropagation</strong><br>
Backpropagation is a fast algorithm for computing the gradient of the cost function.<br>
The heart of the backpropagation is an expression for the partial derivative $\partial C/\partial w$ of the cost function $C$ with respect to any weight $w$ ( or bias $b$) in the network. The expression tells us how quickly the cost changes when change the weights, $w$ and biases, $b$.

### Warm up: a fast matrix-based approach to computing the output from a neural network<br>

<strong>Math</strong><br>
Denote $w^l_{jk}$, the weight for the connection from the $k^{th}$ neuron in the $(l-1)^{th}$ layer to the $j^{th}$ neuron in the $l^{th}$ layer. Look at the diagram below:<br>
![mc1](imgs/mc1.jpg)<br>
This shows the weight from 4th neuron in the 2nd layer to the 2nd neuron in the 3rd layer.<br>
Use the a similar notation for the network's biases and activations. $b_j^l$ for the bias of the $j^{th}$ neuron in the $l^{th}$ layer. And use $a^l_j$ for the activation of the $j^{th}$ neuron in the $l^{th}$ layer. The diagram:<br>
![mc2](imgs/mc2.jpg)<br>
By using these notation we can have the relation:<br>
$$a_j^l = \sigma(\sum_k w^l_{jk}a_k^{l-1} + b_j^l)$$<br>
The sum is over all neurons $k$ in the $(l-1)^{th}$ layer.<br>
Transfer this equation to matrix form. Define the weight matrix $w^l$, the weights connecting to the $l^{th}$ layer, a bias vector, $b^l$ and the activation vector $a^l$. Then transfer equation to matrix form:<br>
$$a^l = \sigma(w^la^{l-1}+b^l)$$<br>
Where the $\sigma$ function is element-wise, $\sigma(v_i)=\sigma(v)_i$.<br>
We also define:<br>
$$z^l\equiv w^la^{l-1}+b^l$$<br>
as <i>weighted input</i> to the neurons in the layer $l$. 
$$a^l = \sigma(z^l)$$

### The two assumptions we need about the $C$<br>
Example cost function:<br>
$$C=\frac{1}{2n}\sum_x\|y(x)-a^L(x)\|^2$$<br>
$n$, total # of training examples; sum over individual training examples, $x$; $y=y(x)$ corresponding desired output; $L$, number of layers in network; $a^L=a^L(x)$, vector of activations output when $x$ is input.<br>
2 assumptions:<br>
1. The cost function can be written as average <br>
$$C=\frac{1}{n}\sum_x C_x$$<br>
$C_x$ is for individual training examples, $x$. In the quadratic cost function, $C_x = \frac{1}{2}\|y-a^L\|^2$.<br>
We need this because the backpropagation actually do the partial derivatives for single training example and then average over all.<br>
2. $C$ can be written as function of the outputs from the neural network:<br>
![asum1](imgs/asum1.jpg)<br>
For the quadratic cost for single training $x$:<br>
$$C=\frac{1}{2}\sum_j(y_j-a_j^L)^2$$<br>
which is a functon of the output activations.

### The Hadamard product, $s \odot t$
Suppose $s$ and $t$ are 2 vectors of the same dimension. Then we use $s \odot t$ to denote the <i>elmentwise</i> product of the two vectors.<br>
$$(s\odot t)_j=s_jt_j$$<br>
$$\begin{bmatrix}1\\2\end{bmatrix}\odot\begin{bmatrix}3\\4\end{bmatrix} = \begin{bmatrix}1*3\\2*4\end{bmatrix} = \begin{bmatrix}3\\8\end{bmatrix}$$<br>

### The 4 fundamental equations behind backpropagation
Define $\delta^l_j$ the error in the $j^{th}$ neuron in the $l^{th}$ layer.<br>
Backpropagation will give the procedure to compute the error and then relate error to the partial derivative part.<br>
To show how, image ther is a demon:<br>
![er1](imgs/er1.jpg)<br>
And define the error $\delta^l_j$ of neuron $j$ in layer $l$ by:<br>
$$\delta^l_j\equiv \frac{\partial C}{\partial z^l_j}$$<br>
As usual $\delta^l$ is all the error vector with layer $l$.<br>

<strong>Plan of attack</strong>:<br>
<strong>An equation for the error in the output layer, $\delta^L$</strong>:<br>
$$\delta^L_j = \frac{\partial C}{\partial a^L_j}\sigma'(z^L_j)$$
If, $C$ doesn't depend much on a particular output neuron, $j$, then $\delta_j^L$ will be small, which is what we expect.<br>
Transfer to matrix form:<br>
$$\delta^L=\nabla_aC\odot\sigma'(z^L)$$<br>
For the quadratic cost, we have $\nabla_aC=(a^L-y)$, thus:<br>
$$\delta^L=(a^L-y)\odot\sigma'(z^L)$$<br>

<strong>An equation for the error $\delta'$ in term of the error in the next layer, $\delta^{l+1}$</strong>:<br>
$$\delta^l = ((w^{l+1})^T\delta^{l+1})\odot\sigma'(z^l)$$<br>
Till now, we can compute the error $\delta^l$ for any layer in the network. First compute $\delta^L$, and then we can compute $\delta^{L-1}$, so on.<br>

<strong>An equation for the rate of change of  the cost with respect to any bias in the network</strong>:<br>
$$\frac{\partial C}{\partial b_j^l} = \delta_j^l$$<br>
Same as:<br>
$$\frac{\partial C}{\partial b}=\delta$$
can be understood that $\delta$ is being evaluated at the same neuron as the bias $b$.

<strong>An equation for the rate of change of  the cost with respect to any weight in the network</strong>:<br>
$$\frac{\partial C}{\partial w^l_{jk}}=a_k^{l-1}\delta^l_j$$<br>
same as:<br>
$$\frac{\partial C}{\partial w} = a_{in}\delta_{out}$$<br>
Can understood as $a_{in}$ is the activation of the neuron input to the weight $w$, $\delta_{out}$ is the error of the neuron output from the weight $w$.

### The backpropagation algorithm<br>
Provide us with a way of compting the gradient of the cost function, $C$. In the form of an algorithm:<br>
1. <strong>Input $x$</strong>: Set the corresponding activation $a^1$ for the input layer.<br>
2. <strong>Feedforward</strong>: For each $l=2, 3, \cdots, L$ compute $z^l=w^la^{l-1}+b^l$ and $a^l=\sigma(z^l)$ <br>
3. <strong>Output error $\delta^L$</strong>: Comput the vector $\delta^L=\nabla_aC\odot\sigma'(z^L)$ <br>
4. <strong>Backpropagate the error</strong>: For each $l= L-1, L-2, \cdots, 2$ compute $\delta^l = ((w^{l+1})^T\delta^{l+1}\odot\sigma'(z^l)$<br>
5. <strong>Output</strong>: The gradient of the cost function is given by $\frac{\partial C}{\partial w^l_{jk}}=a^{l-1}_k\delta^l_j$ and $\frac{\partial C}{\partial b_j^l} = \delta^l_j$ <br>


Algorithm with a mini-batch, m training examples and gradient descent learning step:<br>
1. <strong>Input a set of training examples</strong><br>
2. <strong>For each training example $x$</strong>: Set the corresponding input activation $a^{x,1}$, and perform the following steps:<br>
    (1)<strong>Feedforward</strong>: For each $l=2, 3, \cdots, L$ compute $z^{x,l}=w^la^{x,l-1}+b^l$ and $a^{x,l}=\sigma(z^{x,l})$ <br>
    (2)<strong>Output error $\delta^{x,L}$</strong>: Compute the vector $\delta^{x,L} = \nabla_aC_x\odot\sigma'(z^{x,L})$<br>
    (3)<strong>Backpropagate the error</strong>: For each $l = L-1, L-2,\cdots,2 $ compute $\delta^{x,l}=((w^{l+1})^T\delta^{x,l+1})\odot\sigma'(z^{x,l})$<br>
3. <strong>Gradient descent</strong>: For each $l=L, L-1, \cdots, 2$ update the weigts according to the rule $w^l \rightarrow (w^l)' -\frac{\eta}{m}\sum_x\delta^{x,l}(a^{x,l-1})^T$, and the biases according to the rule $b^l \rightarrow (b^l)' -\frac{\eta}{m}\sum_x\delta^{x,l}$ <br>

To implement the SGD in practive need the outer loop for mini-batches and another outer stepping through multiple epochs of training.<br>

### Backpropagation: the big picture
By made a small change $\Delta w^l_{jk}$ to some weight in the network, $w^l_{jk}$:<br>
![bpp1](imgs/bpp1.jpg)<br>
The change in weight will cause a change in the output activation from the corresponding neuron:<br>
![bpp2](imgs/bpp2.jpg)<br>
Then cause the change in all the activations in the next layer:<br>
![bpp3](imgs/bpp3.jpg)<br>
Then cause changes in the next layer, next, until final layer, and then cost function.<br>
$$\Delta a^l_j \approx \frac{\partial a^l_j}{\partial w^l_{jk}}\Delta w^l_{jk}$$<br>
![bpp4](imgs/bpp4.jpg)<br>
The change in the cost $\Delta C$ is related to the change of the weight, $\Delta w_{jk}^l$ :<br>
$$\Delta C \approx \frac{\partial C}{\partial w^l_{jk}}\Delta w^l_{jk}$$